### Exemple d'utilisation du modèle xgboost avec dataset ravdess

In [ ]:
import librosa
import numpy as np
import joblib
import pandas as pd

xgb_model = joblib.load("model_save/xgb_model.joblib")

# Labels (l'ordre est important)
emotion_labels = ['neutral', 'calm', 'joy', 'sadness', 'anger', 'fear', 'disgust', 'surprise']

# Fonction MFCC
def extract_mfcc(filepath, n_mfcc=40):
    y, sr = librosa.load(filepath, sr=None)
    return np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc).T, axis=0)

# Fichier audio
audio_path = "dataset_clean/test/audios/dia0_utt0.wav"

# Prédiction
features = extract_mfcc(audio_path).reshape(1, -1)
proba = xgb_model.predict_proba(features)[0]
pred_emotion = emotion_labels[np.argmax(proba)]

print(f"\n🎧 Fichier : {audio_path}")
print(f"🔮 Emotion prédite : {pred_emotion}\n")

df = pd.DataFrame({
    "Émotion": emotion_labels,
    "Probabilité": np.round(proba, 4)
})

print("\n📊 Probabilités par émotion :\n")
print(df.to_string(index=False))

**Petite précision : il y a l'émotion "calm" dans le dataset ravdess mais pas dans meld. 
Si c'est un problème on peut par exemple la remplacer par "neutral" avec ce code :** 

In [ ]:
import librosa
import numpy as np
import joblib
import pandas as pd

xgb_model = joblib.load("model_save/xgb_model.joblib")

emotion_labels = ['neutral', 'calm', 'joy', 'sadness', 'anger', 'fear', 'disgust', 'surprise']

def extract_mfcc(filepath, n_mfcc=40):
    y, sr = librosa.load(filepath, sr=None)
    return np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc).T, axis=0)

audio_path = "dataset_clean/test/audios/dia0_utt0.wav"

features = extract_mfcc(audio_path).reshape(1, -1)
proba = xgb_model.predict_proba(features)[0]

# Somme des probabilités 'neutral' + 'calm'
neutral_idx = emotion_labels.index('neutral')
calm_idx = emotion_labels.index('calm')

proba_merged = proba.copy()
proba_merged[neutral_idx] += proba_merged[calm_idx]

# On retire 'calm' du tableau pour éviter doublons
merged_labels = [e for e in emotion_labels if e != 'calm']
merged_proba = np.delete(proba_merged, calm_idx)

# Prédiction sur les probabilités fusionnées
pred_idx = np.argmax(merged_proba)
pred_emotion = merged_labels[pred_idx]

print(f"\n🎧 Fichier : {audio_path}")
print(f"🔮 Emotion prédite : {pred_emotion}\n")

df = pd.DataFrame({
    "Émotion": merged_labels,
    "Probabilité": np.round(merged_proba, 4)
})

print("\n📊 Probabilités par émotion (calm fusionné avec neutral) :\n")
print(df.to_string(index=False))


### En temps réel

In [ ]:
import speech_recognition as sr
import time
import numpy as np
import joblib
import librosa
import pandas as pd

# Charger modèle XGBoost
xgb_model = joblib.load("model_save/xgb_model.joblib")

# Étiquettes émotions (ordre important)
emotion_labels = ['neutral', 'calm', 'joy', 'sadness', 'anger', 'fear', 'disgust', 'surprise']

# Fonction extraction MFCC moyenne depuis buffer audio raw
def extract_mfcc_from_audio_data(audio_data, sr=16000, n_mfcc=40):
    # audio_data est un array numpy float32 de la forme (samples,)
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc.T, axis=0)
    return mfcc_mean

# Callback pour le recognizer (avec transcription)
def callback(recognizer, audio):
    try:
        # Transcription en français
        text = recognizer.recognize_google(audio, language="fr-FR")
        print(f"\n🗣️ Transcription : {text}")

        # Récupérer le buffer audio raw (int16)
        audio_data_int16 = np.frombuffer(audio.get_raw_data(), dtype=np.int16)
        # Convertir en float32 entre -1 et 1
        audio_data = audio_data_int16.astype(np.float32) / 32768.0
        
        # Extraire features MFCC
        features = extract_mfcc_from_audio_data(audio_data).reshape(1, -1)
        
        # Prédire émotion
        proba = xgb_model.predict_proba(features)[0]
        pred_emotion = emotion_labels[np.argmax(proba)]
        
        # Afficher résultats émotion
        print(f"🔮 Emotion prédite : {pred_emotion}")
        df = pd.DataFrame({
            "Émotion": emotion_labels,
            "Probabilité": np.round(proba, 4)
        })
        print(df.to_string(index=False))

    except sr.UnknownValueError:
        print("🛑 Audio incompris.")
    except sr.RequestError as e:
        print(f"❌ Erreur service reconnaissance vocale : {e}")
    except Exception as e:
        print("Erreur lors de l'extraction/prediction :", e)

# Initialisation recognizer et micro
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster pour bruit ambiant
with m as source:
    print("Calibrage du bruit ambiant...")
    r.adjust_for_ambient_noise(source, duration=2)
print("Démarrage de la reconnaissance vocale et détection d'émotion...")

# Lancement de l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

try:
    while True:
        time.sleep(0.1)  # garder le script actif
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Programme arrêté.")
